In [ ]:
import os 
try:
    os.remove('airline2.db')
except OSError:
    pass

In [31]:
import sqlite3
conn = sqlite3.connect('airline2.db')

c = conn.cursor()
import pandas as pd

In [32]:
# sqlite_master is a built-in SQLite metadata table
c.execute('''
SELECT name 
FROM sqlite_master 
WHERE type='table'
''')

In [33]:
c.fetchall()

[('airports',), ('carriers',), ('planes',), ('ontime',)]

In [ ]:
year2000 = pd.read_csv("2000.csv", low_memory=False)
year2001 = pd.read_csv("2001.csv", low_memory=False)
year2002 = pd.read_csv("2002.csv", low_memory=False)
year2003 = pd.read_csv("2003.csv", low_memory=False)
year2004 = pd.read_csv("2004.csv", low_memory=False)
year2005 = pd.read_csv("2005.csv", low_memory=False)
airports = pd.read_csv("airports.csv", low_memory=False)
carriers = pd.read_csv("carriers.csv", low_memory=False)
planes = pd.read_csv("plane-data.csv", low_memory=False)

In [ ]:
year2000.to_sql('ontime_2000', con = conn, index = False)
year2001.to_sql('ontime_2001', con = conn, index = False) 
year2002.to_sql('ontime_2002', con = conn, index = False) 
year2003.to_sql('ontime_2003', con = conn, index = False) 
year2004.to_sql('ontime_2004', con = conn, index = False) 
year2005.to_sql('ontime_2005', con = conn, index = False)
airports.to_sql('airports', con = conn, index = False)
carriers.to_sql('carriers', con = conn, index = False)
planes.to_sql('planes', con = conn, index = False)

In [ ]:
#Create a new table "ontime" to store all 6 csv files

c.execute('''CREATE TABLE ontime(Year INTEGER, Month INTEGER, DayofMonth INTEGER, DayofWeek INTEGER, DepTime TEXT, CRSDepTime INTEGER, ArrTime TEXT, CRSArrTime INTEGER, UniqueCarrier TEXT, FlightNum INTEGER, TailNum TEXT, ActualElapsedTime TEXT, CRSElapsedTime INTEGER, AirTime TEXT, ArrDelay TEXT, DepDelay TEXT, Origin TEXT, Dest TEXT, Distance INTEGER, TaxiIn INTEGER, TaxiOut INTEGER, Cancelled INTEGER, CancellationCode TEXT, Diverted INTEGER, CarrierDelay TEXT, WeatherDelay TEXT, NASDelay TEXT, SecurityDelay TEXT, LateAircraftDelay TEXT)''')
conn.commit()

In [ ]:
c.execute('''
SELECT TOP 0 *
INTO ontime
FROM ontime_2000
''')

In [ ]:
c.execute('''
INSERT INTO ontime
SELECT * FROM ontime_2000''')

c.execute('''
INSERT INTO ontime
SELECT * FROM ontime_2001''')

c.execute('''
INSERT INTO ontime
SELECT * FROM ontime_2002''')

c.execute('''
INSERT INTO ontime
SELECT * FROM ontime_2003''')

c.execute('''
INSERT INTO ontime
SELECT * FROM ontime_2004''')

c.execute('''
INSERT INTO ontime
SELECT * FROM ontime_2005''')


In [ ]:
c.execute('''DROP TABLE ontime_2000''')
c.execute('''DROP TABLE ontime_2001''')
c.execute('''DROP TABLE ontime_2002''')
c.execute('''DROP TABLE ontime_2003''')
c.execute('''DROP TABLE ontime_2004''')
c.execute('''DROP TABLE ontime_2005''')

In [34]:
conn.close()

In [ ]:
#Q1 plane model with lowest associated avg departure delay, excluding cancelled and diverted flights
#model 737-2Y5 avg delay 7.022
q = c.execute('''
SELECT model AS model, AVG(ontime.DepDelay) AS avg_delay
FROM planes INNER JOIN ontime USING(tailnum)
WHERE ontime.Cancelled = 0 AND ontime.Diverted = 0 AND ontime.DepDelay > 0
GROUP BY planes.model
ORDER BY avg_delay
''').fetchall()

pd.DataFrame(q)

In [ ]:
#Q2 city with highest number of inbound flights, excluding cancelled flights
#Chicago with 2388365 inbound flights
q2 = c.execute('''
SELECT airports.city AS city, COUNT(*) AS total
FROM airports INNER JOIN ontime ON ontime.dest = airports.iata
WHERE ontime.Cancelled = 0 
GROUP BY airports.city
ORDER BY total DESC
''').fetchall()

pd.DataFrame(q2)

In [ ]:
#Q3 carrier with highest number of cancelled flights
#Delta Air Lines Inc. with 107851 cancelled flights
q3 = c.execute('''
SELECT carriers.Description AS carrier, COUNT(*) AS total
FROM carriers INNER JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1
GROUP BY carriers.Description
ORDER BY total DESC
''').fetchall()

pd.DataFrame(q3)

In [ ]:
#Q4 carrier with the highest number of cancelled flights, relative to their number of total flights
q4 = c.execute('''
SELECT q1.carrier AS carrier, (CAST(q1.numerator AS FLOAT)/ CAST(q2.denominator AS FLOAT)) AS ratio
FROM (SELECT carriers.Description AS carrier, COUNT(*) AS numerator
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1 
GROUP BY carriers.Description
) AS q1 JOIN
(SELECT carriers.Description AS carrier, COUNT(*) AS denominator
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
GROUP BY carriers.Description
) AS q2 USING(carrier)
ORDER BY ratio DESC
''').fetchall()

pd.DataFrame(q4)

In [ ]:
c